In [1]:
import requests, StringIO, pandas as pd, json, re

In [2]:
def get_file_content(credentials):
    """For given credentials, this functions returns a StringIO object containing the file content."""

    url1 = ''.join([credentials['auth_url'], '/v3/auth/tokens'])
    data = {'auth': {'identity': {'methods': ['password'],
            'password': {'user': {'name': credentials['username'],'domain': {'id': credentials['domain_id']},
            'password': credentials['password']}}}}}
    headers1 = {'Content-Type': 'application/json'}
    resp1 = requests.post(url=url1, data=json.dumps(data), headers=headers1)
    resp1_body = resp1.json()
    for e1 in resp1_body['token']['catalog']:
        if(e1['type']=='object-store'):
            for e2 in e1['endpoints']:
                if(e2['interface']=='public'and e2['region']==credentials['region']):
                    url2 = ''.join([e2['url'],'/', credentials['container'], '/', credentials['filename']])
    s_subject_token = resp1.headers['x-subject-token']
    headers2 = {'X-Auth-Token': s_subject_token, 'accept': 'application/json'}
    resp2 = requests.get(url=url2, headers=headers2)
    return StringIO.StringIO(resp2.content)

In [23]:
content_string = get_file_content(credentials_2)
housing_df = pd.read_csv(content_string)

In [6]:
housing_df.head()

,CONTROL,AGE1,METRO3,REGION,LMED,FMR,L30,L50,L80,IPOV,...,FMTINCRELFMRCAT,FMTCOST06RELAMICAT,FMTCOST08RELAMICAT,FMTCOST12RELAMICAT,FMTCOSTMEDRELAMICAT,FMTINCRELAMICAT,FMTASSISTED,FMTBURDEN,FMTREGION,FMTSTATUS
0,100003130103',82,3',1',73738,956,15738,26213,40322,11067,...,1 LTE 50+ACU- FMR',2 30 +AC0- 50+ACU- AMI',2 30 +AC0- 50+ACU- AMI',2 30 +AC0- 50+ACU- AMI',2 30 +AC0- 50+ACU- AMI',2 30 +AC0- 50+ACU- AMI',.',2 30+ACU- to 50+ACU-',+AC0-5',+AC0-5'
1,100006110249',50,5',3',55846,1100,17165,28604,45744,24218,...,3 GT FMR',4 60 +AC0- 80+ACU- AMI',4 60 +AC0- 80+ACU- AMI',6 100 +AC0- 120+ACU- AMI',4 60 +AC0- 80+ACU- AMI',7 120+ACU- AMI ',.',1 Less than 30+ACU-',+AC0-5',+AC0-5'
2,100006370140',53,5',3',55846,1100,13750,22897,36614,15470,...,2 50.1 +AC0- 100+ACU- FMR',4 60 +AC0- 80+ACU- AMI',5 80 +AC0- 100+ACU- AMI',6 100 +AC0- 120+ACU- AMI',4 60 +AC0- 80+ACU- AMI',4 60 +AC0- 80+ACU- AMI',.',3 50+ACU- or More',+AC0-5',+AC0-5'
3,100006520140',67,5',3',55846,949,13750,22897,36614,13964,...,2 50.1 +AC0- 100+ACU- FMR',6 100 +AC0- 120+ACU- AMI',7 120+ACU- AMI ',7 120+ACU- AMI ',5 80 +AC0- 100+ACU- AMI',4 60 +AC0- 80+ACU- AMI',.',1 Less than 30+ACU-',+AC0-5',+AC0-5'
4,100007130148',26,1',3',60991,737,14801,24628,39421,15492,...,3 GT FMR',3 50 +AC0- 60+ACU- AMI',3 50 +AC0- 60+ACU- AMI',3 50 +AC0- 60+ACU- AMI',3 50 +AC0- 60+ACU- AMI',7 120+ACU- AMI ',0 Not Assisted',1 Less than 30+ACU-',+AC0-5',+AC0-5'


In [12]:
%matplotlib inline

In [22]:
credentials_2 = {
  'auth_url':'https://identity.open.softlayer.com',
  'project':'object_storage_cb2d0fde_b502_47a8_a001_8fe79c4bcc4d',
  'project_id':'31b17e16caeb40dfb46827b48acf9698',
  'region':'dallas',
  'user_id':'b2fc779b212443da96c1e1026ca7eedd',
  'domain_id':'0e7f87b0fd6c4e9c9d1cdffda4555baf',
  'domain_name':'1141683',
  'username':'admin_f33dd542d9bbf8c19cd250880066bc993ed256ed',
  'password':"""Qre02GIJ1..~gs^H""",
  'filename':'thads2013n (2).csv',
  'container':'notebooks',
  'tenantId':'se31-dedd8afad761f5-912ba59f22f5'
}

In [24]:
housing_df.head()

,CONTROL,AGE1,METRO3,LMED,IPOV,ZINC2,ZSMHC
0,740295700149',30,2',63068,15946,14987,0
1,287730160141',30,2',62200,27539,9974,97
2,685890150141',30,2',74316,23581,770,130
3,101267970141',30,2',86700,15992,15000,246
4,300963330103',30,2',70724,15492,51974,317


In [ ]:
mhc = housing_df.ix['ZSMHC']
ax = mhc.plot(kind='bar', figsize=(8,5), title="Monthly Housing Costs Region 4, Metro 2,3" % germany.name)
ax.set_ylabel("Precipitation (million cubic meters)")
ax.set_xlabel("AGE");